In [13]:
import pandas as pd
import nltk
import string
import spacy
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import re
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import SpatialDropout1D
from tensorflow.keras.layers import Embedding

df = pd.read_csv('twitter.csv', encoding='latin-1', nrows=10)
# df = pd.read_csv('twitter.csv', encoding='latin-1')
df.head(10)

,ItemID,Sentiment,SentimentText
0,1,0,is so sad for my APL frie...
1,2,0,I missed the New Moon trail...
2,3,1,omg its already 7:30 :O
3,4,0,.. Omgaga. Im sooo im gunna CRy. I'...
4,5,0,i think mi bf is cheating on me!!! ...
5,6,0,or i just worry too much?
6,7,1,Juuuuuuuuuuuuuuuuussssst Chillin!!
7,8,0,Sunny Again Work Tomorrow :-| ...
8,9,1,handed in my uniform today . i miss you ...
9,10,1,hmmmm.... i wonder how she my number @-)


In [14]:
# Load English tokenizer, tagger, parser, NER and word vectors
'''
def RemoveNER(text):
    filtered = []
    tokens = word_tokenize(text)
    nlp = spacy.load("en_core_web_sm")
    for index, var in enumerate(text):
        aux=0
        doc = nlp(text[index])
        for x in tokens:
            if(len(doc.ents)>0):
                if(x==doc.ents[aux].text):
                    if(aux<len(doc.ents)):
                        aux+=1
                else:
                    filtered.append(x)
        print(filtered)
    new_filtered = ' '.join(filtered)
    return new_filtered

df['SentimentText'] = df['SentimentText'].apply(RemoveNER)
#RemoveNER("Justin drift in car in Brazil")
'''

def IdentifyNER(text):
    nlp = spacy.load("en_core_web_sm")
    doc = nlp(text)
    for entity in doc.ents:
        print(entity.text, entity.label_)
        text = text.replace(entity.text, "")
    return text

df['SentimentText'] = df['SentimentText'].apply(lambda x: IdentifyNER(x))

# Show the cleaned text
df.head(10)

APL ORG
New Moon LOC
7:30 TIME
11 CARDINAL
2 CARDINAL
mi bf PERSON
Juuuuuuuuuuuuuuuuussssst Chillin PERSON
Tomorrow DATE
Tonight TIME
today DATE


,ItemID,Sentiment,SentimentText
0,1,0,is so sad for my friend....
1,2,0,I missed the trailer...
2,3,1,omg its already :O
3,4,0,.. Omgaga. Im sooo im gunna CRy. I'...
4,5,0,i think is cheating on me!!! T_T
5,6,0,or i just worry too much?
6,7,1,!!
7,8,0,Sunny Again Work :-| TV
8,9,1,handed in my uniform . i miss you already
9,10,1,hmmmm.... i wonder how she my number @-)


In [15]:
# Clean the text

# Function for clean text
def CleanTxt(text):
    text = re.sub(u'[^a-zA-Z0-9áéíóúÁÉÍÓÚâêîôÂÊÎÔãõÃÕçÇ: ]', '', text)
    return text

# Function for remove stop words
def RemoveStopWords(text):
    tokens = word_tokenize(text)
    tokens = [w.lower() for w in tokens]
    stop_words = set(stopwords.words('english'))
    words=[w for w in tokens if w not in string.punctuation and w not in stop_words]
    
    new_words = ' '.join(words)

    return new_words

# Cleaning the text
df['SentimentText'] = df['SentimentText'].apply(CleanTxt)
df['SentimentText'] = df['SentimentText'].apply(RemoveStopWords)

# Show the cleaned text
df.head(10)
# print(df['SentimentText'][0][0])

,ItemID,Sentiment,SentimentText
0,1,0,sad friend
1,2,0,missed trailer
2,3,1,omg already
3,4,0,omgaga im sooo im gunna cry ive dentist since ...
4,5,0,think cheating tt
5,6,0,worry much
6,7,1,
7,8,0,sunny work tv
8,9,1,handed uniform miss already
9,10,1,hmmmm wonder number


In [16]:
tweet_df = df[['SentimentText','Sentiment']]
tweet_df.head(10)

,SentimentText,Sentiment
0,sad friend,0
1,missed trailer,0
2,omg already,1
3,omgaga im sooo im gunna cry ive dentist since ...,0
4,think cheating tt,0
5,worry much,0
6,,1
7,sunny work tv,0
8,handed uniform miss already,1
9,hmmmm wonder number,1


In [17]:
tweet = df.SentimentText.values
tokenizer = Tokenizer(num_words=5000)
tokenizer.fit_on_texts(tweet)
vocab_size = len(tokenizer.word_index) + 1
encoded_docs = tokenizer.texts_to_sequences(tweet)
padded_sequence = pad_sequences(encoded_docs, maxlen=200)

In [18]:
# print(tokenizer.word_index)
# print(tweet[0])
# print(encoded_docs[0])
print(padded_sequence[0])

[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 3 4]


In [19]:
embedding_vector_length = 32
model = Sequential()
model.add(Embedding(vocab_size, embedding_vector_length,     
                                     input_length=200) )
model.add(SpatialDropout1D(0.25))
model.add(LSTM(50, dropout=0.5, recurrent_dropout=0.5))
model.add(Dropout(0.2))
model.add(Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy',optimizer='adam', 
                           metrics=['accuracy'])
print(model.summary())

Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
If using Keras pass *_constraint arguments to layers.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 200, 32)           1088      
_________________________________________________________________
spatial_dropout1d (SpatialDr (None, 200, 32)           0         
_________________________________________________________________
lstm (LSTM)                  (None, 50)                16600     
_________________________________________________________________
dropout (Dropout)            (None, 50)                0         
_________________________________________________________________
dense (Dense)              

In [20]:
sentiment_label = df.Sentiment.factorize()

history = model.fit(padded_sequence,sentiment_label[0],
                  validation_split=0.2, epochs=20, batch_size=32)

Train on 8 samples, validate on 2 samples
Epoch 1/20
8/8 [==============================] - 1s 115ms/sample - loss: 0.6952 - acc: 0.1250 - val_loss: 0.7007 - val_acc: 0.0000e+00
Epoch 2/20
8/8 [==============================] - 0s 19ms/sample - loss: 0.6954 - acc: 0.5000 - val_loss: 0.7060 - val_acc: 0.0000e+00
Epoch 3/20
8/8 [==============================] - 0s 25ms/sample - loss: 0.6893 - acc: 0.7500 - val_loss: 0.7106 - val_acc: 0.0000e+00
Epoch 4/20
8/8 [==============================] - 0s 21ms/sample - loss: 0.6841 - acc: 0.7500 - val_loss: 0.7154 - val_acc: 0.0000e+00
Epoch 5/20
8/8 [==============================] - 0s 20ms/sample - loss: 0.6776 - acc: 0.7500 - val_loss: 0.7205 - val_acc: 0.0000e+00
Epoch 6/20
8/8 [==============================] - 0s 20ms/sample - loss: 0.6699 - acc: 0.8750 - val_loss: 0.7258 - val_acc: 0.0000e+00
Epoch 7/20
8/8 [==============================] - 0s 21ms/sample - loss: 0.6797 - acc: 0.7500 - val_loss: 0.7313 - val_acc: 0.0000e+00
Epoch 8/20
8

In [21]:
test_word ="This is soo happy"
tw = tokenizer.texts_to_sequences([test_word])
tw = pad_sequences(tw,maxlen=200)
prediction = int(model.predict(tw).round().item())
sentiment_label[1][prediction]

0